### Load module

In [1]:

from transformers import (
    PaliGemmaProcessor,
    PaliGemmaForConditionalGeneration,
)
from transformers.image_utils import load_image
import torch
import os
from PIL import Image

def load_text(fpaths, by_lines=False):
    with open(fpaths, "r") as fp:
        if by_lines:
            return fp.readlines()
        else:
            return fp.read()

def load_prompt(prompt):
    return load_text(f"prompts/{prompt}.txt")

/home/clhsieh/miniconda3/envs/minecraft/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load model

In [2]:
model_name = "google/paligemma2-10b-mix-448"
# default: Load the model on the available device(s)
model = PaliGemmaForConditionalGeneration.from_pretrained(f"/nfs/turbo/coe-stellayu/clhsieh/Minecraft/ckpt/{model_name}", torch_dtype=torch.bfloat16, device_map="auto").eval()
processor = PaliGemmaProcessor.from_pretrained(f"{model_name}")

Loading checkpoint shards: 100%|██████████| 4/4 [00:15<00:00,  3.82s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


### Load data and define task

In [3]:
for image_name in ['easy-0.png']: # ['easy-1.png', 'easy-2.png', 'easy-3.png', 'easy-4.png', 'easy-5.png']: # ['barn_house.webp', 'castle_wall.webp', 'greek_house.webp', 'mg_nest.png', 'japanese_house.webp', 'incinerator.png']:
# image_name = 'greek_house.webp' # barn_house.webp, castle_wall.webp, greek_house.webp, mg_nest.png, japanese_house.webp, incinerator.png
    # image_name = f'easy-{i}.png'
    folder = 'easy'
    img_path = f"/nfs/turbo/coe-stellayu/clhsieh/Minecraft/data/{folder}/{image_name}"
    image_name_without_ext = os.path.splitext(os.path.basename(img_path))[0]

    output_dir = f"output/{model_name}/{folder}"
    os.makedirs(output_dir, exist_ok=True)
    output_file_path = f"{output_dir}/{image_name_without_ext}.txt"
    

    # building_description_system = load_prompt("building_description_system")
    prompt = load_prompt("building_description_query")
    # prompt = "answer en description of the image"
    raw_image = Image.open(img_path)
    # image = load_image(img_path)
    prompt = "<image>" + prompt
    model_inputs = processor(text=prompt, images=raw_image,
                  padding="longest", do_convert_rgb=True, return_tensors="pt").to("cuda")
    # model_inputs = processor(text=prompt, images=image, return_tensors="pt").to(torch.bfloat16).to(model.device)
    # prompt = ""
    # model_inputs = processor(text=prompt, images=image, return_tensors="pt").to(torch.bfloat16).to(model.device)
    input_len = model_inputs["input_ids"].shape[-1]

    with torch.inference_mode():
        generation = model.generate(**model_inputs, max_new_tokens=4000, do_sample=False)
        generation = generation[0][input_len:]
        output_text = processor.decode(generation, skip_special_tokens=True)
        print(output_text)

    # Save the output_text as a txt file with the name of image_name without the file type



In [4]:
# Save the output_text to the file
with open(output_file_path, "w") as file:
    file.write(output_text[0])

from IPython.display import Image, display

# Display the image
display(Image(filename=img_path))

IndexError: string index out of range

### Process task